## Import the libraries

In [53]:
!pip3 install dataframe_image
!pip3 install pycaret
!pip3 install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
%pip install -U scikit-learn pandas numpy matplotlib "pycaret==3.3.2"


  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.7-cp310-cp310-win_amd64.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.utils import resample
from pycaret.classification import *
#import dataframe_image as dfi
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import os
from pathlib import Path


In [56]:
!python --version


Python 3.10.11


##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

#### Setup the model creation with pycaret

In [57]:
df = pd.read_csv('dataset_large_class_2.csv')
df

,raw_sloc,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,smell_label
0,488.0,284.0,104.0,0.0,54.0,57.0,83.0,294.284957,484.129871,5.744681,2781.171600,154.509533,0.161377,large-class
1,179.0,421.0,138.0,0.0,83.0,68.0,125.0,381.426462,760.932855,4.762295,3623.786794,201.321489,0.253644,large-class
2,581.0,1939.0,685.0,0.0,397.0,245.0,596.0,1915.006330,4730.236212,3.294606,15584.263701,865.792428,1.576745,large-class
3,454.0,1672.0,494.0,0.0,273.0,180.0,410.0,1310.581943,3071.659769,4.706897,14457.984777,803.221377,1.023887,large-class
4,693.0,661.0,192.0,0.0,379.0,304.0,582.0,2400.966744,4800.293813,13.345070,64060.258981,3558.903277,1.600098,large-class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8097,8.0,0.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,non-large-class
8098,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,non-large-class
8099,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,non-large-class
8100,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,non-large-class


In [58]:
# SOLUÇÃO 4: Usar RFE (Recursive Feature Elimination) como alternativa
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

print("🔧 Testando diferentes abordagens de feature selection...")

# Separar features e target
X = df.drop('smell_label', axis=1)
y = df['smell_label']

# Codificar target para numérico
y_encoded = y.map({'non-large-class': 0, 'large-class': 1})

print(f"Shape original: {X.shape}")
print(f"Distribuição das classes: {y.value_counts().to_dict()}")

# Testar diferentes números de features
results = []
for n_features in range(3, min(10, X.shape[1]) + 1):
    print(f"\nTestando com {n_features} features...")
    
    # RFE com RandomForest
    rfe = RFE(estimator=RandomForestClassifier(n_estimators=50, random_state=42), 
              n_features_to_select=n_features)
    X_rfe = rfe.fit_transform(X, y_encoded)
    
    # Obter features selecionadas
    selected_features = X.columns[rfe.support_].tolist()
    
    # Avaliar com cross-validation
    rf = RandomForestClassifier(n_estimators=50, random_state=42, class_weight='balanced')
    scores = cross_val_score(rf, X_rfe, y_encoded, cv=5, scoring='f1')
    
    results.append({
        'n_features': n_features,
        'features': selected_features,
        'mean_f1': scores.mean(),
        'std_f1': scores.std()
    })
    
    print(f"  Features: {selected_features}")
    print(f"  F1 Score: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Selehar melhor configuração
best_result = max(results, key=lambda x: x['mean_f1'])
print(f"\n🏆 Melhor configuração:")
print(f"  Número de features: {best_result['n_features']}")
print(f"  Features: {best_result['features']}")
print(f"  F1 Score: {best_result['mean_f1']:.4f}")

# Criar dataset otimizado
df_optimized = df[best_result['features'] + ['smell_label']].copy()
print(f"\nDataset otimizado criado: {df_optimized.shape}")
print("\nUsando este dataset no setup do PyCaret...")

# Setup com dataset otimizado
s = setup(
    data=df_optimized,
    target='smell_label',
    session_id=42,
    fold=10,
    fix_imbalance=True,
    fix_imbalance_method=SMOTE(k_neighbors=3, random_state=42),
    feature_selection=False,  # Já aplicamos RFE
    remove_multicollinearity=False,  # Já controlamos no RFE
    normalize=True,
    transformation=True,
    verbose=False
)

🔧 Testando diferentes abordagens de feature selection...
Shape original: (8102, 13)
Distribuição das classes: {'non-large-class': 8063, 'large-class': 39}

Testando com 3 features...
  Features: ['raw_sloc', 'raw_blank', 'hal_func_calculated_length']
  F1 Score: nan (+/- nan)

Testando com 4 features...
  Features: ['raw_sloc', 'raw_blank', 'hal_func_vocabulary', 'hal_func_calculated_length']
  F1 Score: nan (+/- nan)

Testando com 5 features...
  Features: ['raw_sloc', 'raw_multi', 'raw_blank', 'hal_func_vocabulary', 'hal_func_calculated_length']
  F1 Score: nan (+/- nan)

Testando com 6 features...
  Features: ['raw_sloc', 'raw_multi', 'raw_blank', 'hal_func_vocabulary', 'hal_func_calculated_length', 'hal_func_bugs']
  F1 Score: nan (+/- nan)

Testando com 7 features...
  Features: ['raw_sloc', 'raw_multi', 'raw_blank', 'hal_func_N2', 'hal_func_vocabulary', 'hal_func_calculated_length', 'hal_func_bugs']
  F1 Score: nan (+/- nan)

Testando com 8 features...
  Features: ['raw_sloc', 'r

In [59]:
# SOLUÇÃO 2: Aplicar feature selection manualmente ANTES do setup para evitar o erro
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

print("🔧 Aplicando feature selection manualmente para evitar o IndexError...")

# Separar features e target
X = df.drop('smell_label', axis=1)
y = df['smell_label']

# 1. Remover manualmente features com correlação > 0.95
print("Passo 1: Removendo features altamente correlacionadas...")
corr_matrix = X.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print(f"Features removidas por alta correlação: {to_drop}")
X_reduced = X.drop(columns=to_drop)

print(f"Shape antes: {X.shape} -> Shape depois: {X_reduced.shape}")

# 2. Aplicar feature selection com SelectKBest
print("Passo 2: Aplicando SelectKBest...")
selector = SelectKBest(score_func=f_classif, k=min(8, X_reduced.shape[1]))  # Selecionar top 8 features
X_selected = selector.fit_transform(X_reduced, y)

# Obter nomes das features selecionadas
selected_features = X_reduced.columns[selector.get_support()].tolist()
print(f"Features selecionadas: {selected_features}")

# 3. Criar novo dataframe com features selecionadas
df_processed = df[selected_features + ['smell_label']].copy()
print(f"Shape final do dataset processado: {df_processed.shape}")
print("\nDistribuição das classes:")
print(df_processed['smell_label'].value_counts())

# 4. Setup do PyCaret com o dataset já processado (sem feature_selection para evitar conflito)
s = setup(
    data=df_processed,
    target='smell_label',
    session_id=42,
    fold=10,
    fix_imbalance=True,
    fix_imbalance_method=SMOTE(k_neighbors=5, random_state=42),
    feature_selection=False,  # Já aplicamos manualmente
    remove_multicollinearity=False,  # Já removemos manualmente
    normalize=True,
    transformation=True,
    verbose=False
)

🔧 Aplicando feature selection manualmente para evitar o IndexError...
Passo 1: Removendo features altamente correlacionadas...
Features removidas por alta correlação: ['raw_blank', 'hal_func_vocabulary', 'hal_func_length', 'hal_func_calculated_length', 'hal_func_volume', 'hal_func_effort', 'hal_func_time', 'hal_func_bugs']
Shape antes: (8102, 13) -> Shape depois: (8102, 5)
Passo 2: Aplicando SelectKBest...
Features selecionadas: ['raw_sloc', 'raw_multi', 'raw_single_comments', 'hal_func_N2', 'hal_func_difficulty']
Shape final do dataset processado: (8102, 6)

Distribuição das classes:
smell_label
non-large-class    8063
large-class          39
Name: count, dtype: int64


In [60]:
dft = get_config('X_train').shape[1]
dft

5

In [61]:
df = get_config('X')
df

,raw_sloc,raw_multi,raw_single_comments,hal_func_N2,hal_func_difficulty
751,2.0,0.0,0.0,0.0,0.000000
84,28.0,0.0,0.0,21.0,3.315789
8065,12.0,0.0,0.0,8.0,1.500000
2942,3.0,0.0,0.0,0.0,0.000000
7772,13.0,0.0,0.0,2.0,0.500000
...,...,...,...,...,...
6332,11.0,0.0,0.0,2.0,0.500000
4324,2.0,3.0,0.0,0.0,0.000000
7902,29.0,12.0,1.0,18.0,2.250000
978,18.0,0.0,0.0,2.0,0.500000


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [62]:
top5_models = compare_models(n_select=13, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9944,0.9331,0.9944,0.9960,0.9950,0.5462,0.5673,0.0680
knn,K Neighbors Classifier,0.9908,0.8548,0.9908,0.9953,0.9928,0.4537,0.4857,0.1920
ridge,Ridge Classifier,0.9139,0.8609,0.9139,0.9943,0.9506,0.0714,0.1688,0.0100
lda,Linear Discriminant Analysis,0.9139,0.8608,0.9139,0.9943,0.9506,0.0715,0.1688,0.2550
svm,SVM - Linear Kernel,0.9138,0.8564,0.9138,0.9939,0.9505,0.0635,0.1503,0.0120
lr,Logistic Regression,0.9011,0.8573,0.9011,0.9941,0.9434,0.0561,0.1457,0.2660
nb,Naive Bayes,0.8801,0.8557,0.8801,0.9942,0.9317,0.0490,0.1390,0.0110
qda,Quadratic Discriminant Analysis,0.8771,0.8339,0.8771,0.9940,0.9300,0.0455,0.1289,0.0110
dummy,Dummy Classifier,0.0048,0.5000,0.0048,0.0000,0.0000,0.0000,0.0000,0.0100


In [63]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('Multilabel_geral.latex',index=False)
df3.to_csv('Multilabel_geral.csv',index=False)
#dfi.export(df2, 'Bin_Blob_FS1.png')

##### Result by Smell

In [64]:
from sklearn.metrics import classification_report
from pycaret.classification import predict_model

# Dicionário para armazenar os relatórios por modelo e por classe
reports = {}
# Lista para armazenar os dados
data = []

# Iterar sobre os modelos
for model in top5_models:
    # Gerar predições com o modelo individual
    predictions = predict_model(model)

    # Obter nome do modelo
    model_name = model.__class__.__name__

    # Gerar relatório por classe
    report = classification_report(
        predictions['smell_label'],
        predictions['prediction_label'],
        output_dict=True
    )

    # Armazenar o relatório
    reports[model_name] = report

    # Exibir e armazenar métricas por classe
    print(f"\n📊 Classification Report for {model_name}")
    for smell_class in ['large-class']:
        if smell_class in report:
            print(f"\nSmell: {smell_class}")
            print(f"  Accuracy: {report['accuracy']:.2f}")
            print(f"  Precision: {report[smell_class]['precision']:.2f}")
            print(f"  Recall: {report[smell_class]['recall']:.2f}")
            print(f"  F1-score: {report[smell_class]['f1-score']:.2f}")

            # Armazenar para eventual DataFrame
            data.append({
                "Model": model_name,
                "Smell": smell_class,
                "Accuracy": report["accuracy"],
                "Precision": report[smell_class]["precision"],
                "Recall": report[smell_class]["recall"],
                "F1-score": report[smell_class]["f1-score"]
            })
    print("=" * 80)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9955,0.7584,0.9955,0.9957,0.9956,0.5577,0.5582



📊 Classification Report for LGBMClassifier

Smell: large-class
  Accuracy: 1.00
  Precision: 0.54
  Recall: 0.58
  F1-score: 0.56


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9893,0.8296,0.9893,0.9947,0.9916,0.3766,0.4174



📊 Classification Report for KNeighborsClassifier

Smell: large-class
  Accuracy: 0.99
  Precision: 0.27
  Recall: 0.67
  F1-score: 0.38


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ridge Classifier,0.9111,0,0.9111,0.9934,0.9490,0.0602,0.1414


...


📊 Classification Report for GaussianNB

Smell: large-class
  Accuracy: 0.87
  Precision: 0.03
  Recall: 0.67
  F1-score: 0.05


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.8778,0.7634,0.8778,0.9933,0.9303,0.0420,0.1161



📊 Classification Report for QuadraticDiscriminantAnalysis

Smell: large-class
  Accuracy: 0.88
  Precision: 0.03
  Recall: 0.67
  F1-score: 0.05


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Dummy Classifier,0.0049,0.5000,0.0049,0.0000,0.0000,0.0000,0.0000



📊 Classification Report for DummyClassifier

Smell: large-class
  Accuracy: 0.00
  Precision: 0.00
  Recall: 1.00
  F1-score: 0.01


In [65]:
# Criar um DataFrame
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv("smell_classification_Multilabel.csv", index=False)

In [66]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('large_class_.latex',index=False)
df3.to_csv('large_class.csv',index=False)

In [67]:
top_models = compare_models(n_select=13, sort='f1')  
best = top_models[0]
best


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9944,0.9331,0.9944,0.9960,0.9950,0.5462,0.5673,0.0600
knn,K Neighbors Classifier,0.9908,0.8548,0.9908,0.9953,0.9928,0.4537,0.4857,0.0190
ridge,Ridge Classifier,0.9139,0.8609,0.9139,0.9943,0.9506,0.0714,0.1688,0.0100
lda,Linear Discriminant Analysis,0.9139,0.8608,0.9139,0.9943,0.9506,0.0715,0.1688,0.2610
svm,SVM - Linear Kernel,0.9138,0.8564,0.9138,0.9939,0.9505,0.0635,0.1503,0.0130
lr,Logistic Regression,0.9011,0.8573,0.9011,0.9941,0.9434,0.0561,0.1457,0.0130
nb,Naive Bayes,0.8801,0.8557,0.8801,0.9942,0.9317,0.0490,0.1390,0.0110
qda,Quadratic Discriminant Analysis,0.8771,0.8339,0.8771,0.9940,0.9300,0.0455,0.1289,0.0180
dummy,Dummy Classifier,0.0048,0.5000,0.0048,0.0000,0.0000,0.0000,0.0000,0.0110


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

#### Ensemble the top-5 models

In [68]:
# Afinar o melhor:
tuned = tune_model(best, optimize='f1', choose_better=True)

# Pegar um conjunto mais diverso para blend:
top5 = compare_models(n_select=5, sort='f1')
blended = blend_models(top5, optimize='f1', choose_better=True)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9982,1.0000,0.9982,0.9987,0.9984,0.8563,0.8653
1,0.9947,0.9973,0.9947,0.9979,0.9958,0.5693,0.6308
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9982,1.0000,0.9982,0.9988,0.9984,0.7991,0.8158
4,0.9982,1.0000,0.9982,0.9987,0.9984,0.8563,0.8653
5,0.9965,0.9376,0.9965,0.9965,0.9956,0.4987,0.5763
6,0.9929,0.9764,0.9929,0.9951,0.9938,0.4967,0.5132
7,0.9894,0.7911,0.9894,0.9922,0.9907,0.2450,0.2532
8,0.9929,1.0000,0.9929,0.9970,0.9943,0.5970,0.6523


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9944,0.9331,0.9944,0.9960,0.9950,0.5462,0.5673,0.0890
knn,K Neighbors Classifier,0.9908,0.8548,0.9908,0.9953,0.9928,0.4537,0.4857,0.0190
ridge,Ridge Classifier,0.9139,0.8609,0.9139,0.9943,0.9506,0.0714,0.1688,0.0110
lda,Linear Discriminant Analysis,0.9139,0.8608,0.9139,0.9943,0.9506,0.0715,0.1688,0.2580
svm,SVM - Linear Kernel,0.9138,0.8564,0.9138,0.9939,0.9505,0.0635,0.1503,0.0120
lr,Logistic Regression,0.9011,0.8573,0.9011,0.9941,0.9434,0.0561,0.1457,0.0110
nb,Naive Bayes,0.8801,0.8557,0.8801,0.9942,0.9317,0.0490,0.1390,0.0110
qda,Quadratic Discriminant Analysis,0.8771,0.8339,0.8771,0.9940,0.9300,0.0455,0.1289,0.0120
dummy,Dummy Classifier,0.0048,0.5000,0.0048,0.0000,0.0000,0.0000,0.0000,0.0100


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9208,0.0000,0.9208,0.9950,0.9541,0.1088,0.2398
1,0.9312,0.0000,0.9312,0.9966,0.9612,0.0869,0.2131
2,0.9030,0.0000,0.9030,0.9966,0.9457,0.0614,0.1780
3,0.9171,0.0000,0.9171,0.9966,0.9535,0.0721,0.1934
4,0.9030,0.0000,0.9030,0.9950,0.9442,0.0892,0.2161
5,0.9559,0.0000,0.9559,0.9913,0.9726,0.0653,0.1054
6,0.9295,0.0000,0.9295,0.9931,0.9587,0.0819,0.1674
7,0.9506,0.0000,0.9506,0.9912,0.9698,0.0577,0.0978
8,0.9365,0.0000,0.9365,0.9951,0.9627,0.1344,0.2684


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [69]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9014,0.0000,0.9014,0.9950,0.9434,0.0876,0.2140
1,0.9083,0.0000,0.9083,0.9966,0.9487,0.0651,0.1834
2,0.8942,0.0000,0.8942,0.9966,0.9408,0.0560,0.1698
3,0.9083,0.0000,0.9083,0.9966,0.9487,0.0651,0.1834
4,0.8959,0.0000,0.8959,0.9950,0.9403,0.0831,0.2081
5,0.9400,0.0000,0.9400,0.9931,0.9644,0.0965,0.1833
6,0.9065,0.0000,0.9065,0.9930,0.9461,0.0608,0.1420
7,0.9418,0.0000,0.9418,0.9912,0.9651,0.0479,0.0875
8,0.8977,0.0000,0.8977,0.9950,0.9413,0.0845,0.2101


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [70]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9955,0.7584,0.9955,0.9957,0.9956,0.5577,0.5582


In [71]:
result = pull()

In [72]:
result.to_csv('LargeClass_ensemble.csv')

#### Plot the ensemble model

#### Finalize and save the ensemble model

In [73]:
candidate = best  # ou tuned / best

try:
    final_best = finalize_model(candidate)
except Exception as e:
    print("[Aviso] finalize_model falhou. Reaplicando workaround com fix_imbalance=False...")
    set_config('fix_imbalance', False)   # garante que o PyCaret finalize sem reamostragem
    final_best = finalize_model(candidate)

final_best


Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['raw_sloc', 'raw_multi',
                                             'raw_single_comments',
                                             'hal_func_N2',
                                             'hal_func_difficulty'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fi...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None, random_state=42,
                                reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0))],
         verbose=False)

In [74]:
save_model(final_best, 'large_class_sk_without_fs')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['raw_sloc', 'raw_multi',
                                              'raw_single_comments',
                                              'hal_func_N2',
                                              'hal_func_difficulty'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fi...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, importance_type='split',
                            